In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import TheoreticallyOptimalStrategy as TOS
from marketsimcode import compute_portvals, compute_stats
import indicators
from util import get_data

In [18]:
symbol = "JPM"
in_sample_sd = dt.datetime(2008, 1, 1)
in_sample_ed = dt.datetime(2009, 12, 31)
out_sample_sd = dt.datetime(2010, 1, 1)
out_sample_ed = dt.datetime(2011, 12, 31)
sv = 100000
commission = 9.95
impact = 0.05

#MANUAL STRATEGY

In [35]:
# Manual Strategy

indicator = indicators.Indicators(symbol, pd.date_range(sd, ed))
bbp = indicator.get_bbp(window = 20)
rsi = indicator.get_rsi(window = 14)
macd = indicator.get_macd().loc[sd:] # My version of indicators.py is also returning values 9 market days prior to sd

# Calculate MACD crossover
macd_prev = macd.shift(1)
macd_prev.iloc[0] = macd_prev.iloc[1]
conditions = [
    (macd_prev <= 0) & (macd > 0),
    (macd_prev >= 0) & (macd < 0)
]
values = [1, -1]
# Documentation: https://numpy.org/doc/2.1/reference/generated/numpy.select.html
macd_crossover = pd.DataFrame({'result': np.select(conditions, values, default=0)}, index=macd.index, columns=["result"])

# Generate signals
signals_cond = [
    ((bbp <= 0) & (rsi <= 30)) | ((bbp <= 0) & (macd_crossover["result"] == 1)) | ((rsi <= 30) & (macd_crossover["result"] == 1)),
    ((bbp >= 100) & (rsi >= 70)) | ((bbp >= 100) & (macd_crossover["result"] == 1)) | ((rsi >= 70) & (macd_crossover["result"] == 1))
]
signal_values = [1, -1]
signals = pd.DataFrame({'result': np.select(signals_cond, signal_values, default=0)}, index=bbp.index)

# Generate Manual Strategy trades
net_position = 0
manual_trades = pd.DataFrame(0, index=bbp.index, columns=[symbol])

for date in signals.index:
    if signals.loc[date].iloc[0] == 1:
        manual_trades.loc[date] = 1000 - net_position
    elif signals.loc[date].iloc[0] == -1:
        manual_trades.loc[date] = -1000 - net_position

    net_position += manual_trades.loc[pd.to_datetime(date), symbol]

portvals = compute_portvals(manual_trades, start_val=sv, commission = commission, impact=impact)

portvals

2008-01-02    100000.0
2008-01-03    100000.0
2008-01-04     88183.5
2008-01-07     88563.5
2008-01-08     87063.5
                ...   
2008-12-24     41353.5
2008-12-26     41303.5
2008-12-29     41293.5
2008-12-30     42443.5
2008-12-31     42933.5
Length: 253, dtype: float64

In [36]:
compute_stats(portvals)

(-0.570665, -0.0016337761780937103, 0.05524187128897186, -0.46948794792624565)

#STRATEGY LEARNER